In [1]:
import time
import io
import serial
import math
from qiskit import *


ser = serial.Serial('/dev/tty.usbmodem14202', timeout=1, baudrate=115200)
sio = io.TextIOWrapper(io.BufferedRWPair(ser, ser))

print('Let us create cubits and quantum circuit..wait a moment..')
#creating registers, q=quantum, c=classical
qreg = QuantumRegister(3)
creg = ClassicalRegister(3)
circuit = QuantumCircuit(qreg,creg)

circuit.ry(2*math.acos(1/math.sqrt(3)), qreg[0])
circuit.ch(qreg[0], qreg[1])

circuit.h(qreg[2])
circuit.barrier()
circuit.measure(qreg[0], creg[0])
circuit.measure(qreg[1], creg[1])
circuit.measure(qreg[2],creg[2])
#circuit.draw(output='mpl')
print('Quantum circuit created! Ready for simulation of Quantum Circuit.')
simulator = Aer.get_backend('qasm_simulator')
execute(circuit, backend=simulator)
result = execute(circuit, backend=simulator, memory=True).result().get_memory()
print('setup and simulation ready!')
print('Waiting command from microbit')

try:
    #ser.write(b'pong')
    while True:
        serial_in = str(sio.readline())
        print('Found in serial: ' + serial_in)
        if 'start' in serial_in:
            IBMQ.load_account()
            provider = IBMQ.get_provider('ibm-q')
            qcomp = provider.get_backend('ibmq_essex')
            job = execute(circuit, backend=qcomp)
            from qiskit.tools.monitor import job_monitor
            job_monitor(job) # Koodi jää odottamaan tähän, kun lähetys on jonossa ja käsiteltävänä
            result = job.result()
            counts = result.get_counts()
            dice_result_freq = 0
            best_score = 0
            for key in counts:
                if counts[key] > dice_result_freq:
                    dice_result_freq = counts[key]
                    best_score = key
            number = int(best_score,2)
            #plot_histogram(result.get_counts(circuit))
            message ='stop'+str(number)
            ser.write(bytes(message, 'utf-8'))
            #ser.write(b'\n')
            #print('wrote into serial: pong')
            #plot_histogram(result.get_counts(circuit))
            print(result.get_counts())
        time.sleep(0.5)

except KeyboardInterrupt:
    ser.close()
    print('Ohjelman suoritus paattyy.')

Let us create cubits and quantum circuit..wait a moment..
Quantum circuit created! Ready for simulation of Quantum Circuit.
setup and simulation ready!
Waiting command from microbit
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: 
Found in serial: start
Job Status: job has successfully run


TypeError: can't concat str to bytes